<a href="https://colab.research.google.com/github/slyofzero/Movie-Recommender/blob/main/Movie_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Aim-

To create an user liking based movie recommendation system.

---

Data - https://raw.githubusercontent.com/slyofzero/Movie-Recommender/main/ratings.csv

In [169]:
# Loading the dataset.
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

ratings_df = pd.read_csv("https://raw.githubusercontent.com/slyofzero/Movie-Recommender/main/ratings.csv")
ratings_df.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [170]:
# Creating a table with user ratings by each user for each movie.
movie_ratings_matrix = ratings_df.pivot_table(index="user_id",columns="title",values="rating")
movie_ratings_matrix

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
# Viewing the correlation of a random movie with other movies.
movie_corr = movie_ratings_matrix.corr()["\'Til There Was You (1997)"]
movie_corr = movie_corr.sort_values(ascending = False)
movie_corr = movie_corr.dropna()
movie_corr

title
Amistad (1997)                  1.0
'Til There Was You (1997)       1.0
From Dusk Till Dawn (1996)      1.0
Home for the Holidays (1995)    1.0
Home Alone (1990)               1.0
                               ... 
Taxi Driver (1976)             -1.0
Close Shave, A (1995)          -1.0
Chinatown (1974)               -1.0
Celluloid Closet, The (1995)   -1.0
Wrong Trousers, The (1993)     -1.0
Name: 'Til There Was You (1997), Length: 341, dtype: float64

In [89]:
movie_ratings_info_df = ratings_df.groupby("title").agg({"rating":[np.size, np.mean]})
movie_ratings_info_df.columns = ["no. of ratings", "avg score"]

In [90]:
valid_movies = movie_ratings_info_df[movie_ratings_info_df["no. of ratings"] >= 100].index
movie_ratings_matrix = movie_ratings_matrix[valid_movies]

In [96]:
movie_ratings_matrix.corr()

title,101 Dalmatians (1996),12 Angry Men (1957),2001: A Space Odyssey (1968),Absolute Power (1997),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),"Adventures of Priscilla, Queen of the Desert, The (1994)","African Queen, The (1951)",Air Force One (1997),Aladdin (1992),...,Welcome to the Dollhouse (1995),What's Eating Gilbert Grape (1993),When Harry Met Sally... (1989),While You Were Sleeping (1995),William Shakespeare's Romeo and Juliet (1996),Willy Wonka and the Chocolate Factory (1971),"Wizard of Oz, The (1939)","Wrong Trousers, The (1993)",Young Frankenstein (1974),Young Guns (1988)
title,,,,,,,,,,,,,,,,,,,,,
101 Dalmatians (1996),1.000000,-0.049890,-0.043407,0.398783,0.236350,-0.147491,-0.135467,0.192703,0.221943,0.385118,...,-0.110921,0.130558,0.312021,0.021598,0.257404,0.105261,0.366592,-0.002382,0.158840,0.119234
12 Angry Men (1957),-0.049890,1.000000,0.178848,-0.163455,0.045374,-0.243775,0.343750,0.266966,0.228031,0.030291,...,-0.221473,0.115089,0.169496,-0.077648,0.265786,0.149164,0.165508,0.330186,0.096546,0.068944
2001: A Space Odyssey (1968),-0.043407,0.178848,1.000000,-0.241580,0.089206,0.138417,0.077123,0.152433,-0.282994,-0.067992,...,-0.013706,-0.011228,0.023712,-0.050377,0.128939,0.147293,0.236830,0.186458,-0.001307,-0.174918
Absolute Power (1997),0.398783,-0.163455,-0.241580,1.000000,0.122946,-0.147521,-0.130843,0.586266,0.497411,-0.016733,...,0.091940,0.111671,0.156774,0.209359,0.159979,-0.025037,0.122098,-0.260960,0.050316,0.254324
"Abyss, The (1989)",0.236350,0.045374,0.089206,0.122946,1.000000,0.290074,0.214009,0.365772,0.140089,0.364484,...,0.239805,0.242451,0.336413,0.201318,0.609130,0.092120,0.402989,0.139983,0.140161,0.384703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Willy Wonka and the Chocolate Factory (1971),0.105261,0.149164,0.147293,-0.025037,0.092120,0.161218,0.008702,0.054484,-0.132312,0.159210,...,0.234318,-0.027638,0.096966,0.019665,0.177106,1.000000,0.294893,0.036025,0.291210,0.138136
"Wizard of Oz, The (1939)",0.366592,0.165508,0.236830,0.122098,0.402989,0.180994,-0.001315,0.508389,0.073739,0.300988,...,0.363314,0.133906,0.239380,-0.001873,0.000392,0.294893,1.000000,0.026274,0.244063,0.199910
"Wrong Trousers, The (1993)",-0.002382,0.330186,0.186458,-0.260960,0.139983,-0.315877,0.341883,0.287088,0.094060,0.184790,...,0.008269,0.302068,0.213810,0.022222,0.386921,0.036025,0.026274,1.000000,0.116698,-0.019082


In [161]:
user_input = movie_ratings_matrix.loc[0, :].dropna()
user_input

title
Empire Strikes Back, The (1980)    5.0
Gone with the Wind (1939)          1.0
Star Wars (1977)                   5.0
Name: 0, dtype: float64

In [164]:
def movie_recommder(user_input):
  user_input = pd.Series(user_input)
  best_movies = pd.Series()

  for movie in user_input.index:
    highly_corr_movies = movie_ratings_matrix.corr()[movie].dropna()
    highly_corr_movies = highly_corr_movies.sort_values(ascending = False)
    highly_corr_movies = highly_corr_movies * user_input[movie]
    
    best_movies = best_movies.append(highly_corr_movies)

  best_movies = best_movies.sort_values(ascending = False)
  best_movies = best_movies.drop(index = user_input.index).index.unique()[:5]
  
  return best_movies

movie_recommder(user_input)

Index(['Return of the Jedi (1983)', 'Raiders of the Lost Ark (1981)',
       'Philadelphia Story, The (1940)', 'Frighteners, The (1996)',
       'Con Air (1997)'],
      dtype='object')